In [32]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [4]:
#Load the dataset 

df = pd.read_csv("..\Python_Checkpoints\STEG_BILLING_HISTORY.csv")

C:\Users\zakaa\AppData\Local\Temp\ipykernel_43672\2774413690.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("..\Python_Checkpoints\STEG_BILLING_HISTORY.csv")


In [5]:
#Display the ten first lines

client_0_bills =pd.DataFrame(df.head(10))
client_0_bills

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667.0,0,203,8.0,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667.0,0,203,6.0,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667.0,0,203,8.0,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667.0,0,207,8.0,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667.0,0,207,9.0,1,572,0,0,0,15066,15638,12,ELEC
5,train_Client_0,2017-07-17,11,1335667.0,0,207,9.0,1,314,0,0,0,15638,15952,8,ELEC
6,train_Client_0,2018-12-07,11,1335667.0,0,207,9.0,1,541,0,0,0,15952,16493,12,ELEC
7,train_Client_0,2019-03-19,11,1335667.0,0,207,9.0,1,585,0,0,0,16493,17078,8,ELEC
8,train_Client_0,2011-07-22,11,1335667.0,0,203,9.0,1,1200,186,0,0,7770,9156,4,ELEC
9,train_Client_0,2011-11-22,11,1335667.0,0,203,6.0,1,1082,0,0,0,9156,10238,4,ELEC


In [6]:
#Data type of the 'client_0_bills'

client_0_bills.dtypes

client_id                object
invoice_date             object
tarif_type                int64
counter_number          float64
counter_statue           object
counter_code              int64
reading_remarque        float64
counter_coefficient       int64
consommation_level_1      int64
consommation_level_2      int64
consommation_level_3      int64
consommation_level_4      int64
old_index                 int64
new_index                 int64
months_number             int64
counter_type             object
dtype: object

In [8]:
client_0_bills.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             10 non-null     object 
 1   invoice_date          10 non-null     object 
 2   tarif_type            10 non-null     int64  
 3   counter_number        10 non-null     float64
 4   counter_statue        10 non-null     object 
 5   counter_code          10 non-null     int64  
 6   reading_remarque      10 non-null     float64
 7   counter_coefficient   10 non-null     int64  
 8   consommation_level_1  10 non-null     int64  
 9   consommation_level_2  10 non-null     int64  
 10  consommation_level_3  10 non-null     int64  
 11  consommation_level_4  10 non-null     int64  
 12  old_index             10 non-null     int64  
 13  new_index             10 non-null     int64  
 14  months_number         10 non-null     int64  
 15  counter_type          10 n

In [9]:
#Rows and columns

rows, columns = df.shape
print(
    f"The dataset has {rows} rows and {columns} columns."
)

The dataset has 4476749 rows and 16 columns.


In [10]:
# categorical features
client_0_bills.nunique()

client_id                1
invoice_date            10
tarif_type               1
counter_number           1
counter_statue           1
counter_code             2
reading_remarque         3
counter_coefficient      1
consommation_level_1     9
consommation_level_2     3
consommation_level_3     1
consommation_level_4     1
old_index               10
new_index               10
months_number            3
counter_type             1
dtype: int64

In [ ]:
#Memory usage
print(f"The memory consumation is of: {df.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB")   # in MB



The memory consumation is of: 1430.362177848816 MB


In [12]:
#Inspect the dataset for potential missing values

df.isnull().sum()

client_id                  0
invoice_date               0
tarif_type                 0
counter_number            48
counter_statue             0
counter_code               0
reading_remarque        4531
counter_coefficient        0
consommation_level_1       0
consommation_level_2       0
consommation_level_3       0
consommation_level_4       0
old_index                  0
new_index                  0
months_number              0
counter_type               0
dtype: int64

In [13]:
#Strategy on handling missing Values

#Checking Forward and Backward Fill 
missing_rows = df.loc[df[['counter_number', 'reading_remarque']].isna().any(axis=1), ['counter_number', 'reading_remarque']]
missing_rows_indices = missing_rows.index
context_indices = set()

for idx in missing_rows_indices:
    context_indices.update([idx - 1, idx, idx + 1])

context_df = df.loc[df.index.intersection(context_indices),['counter_number', 'reading_remarque']].sort_index()
context_df 
#It seems this Strategy might work, it follows a pattern with specific condition.


""" 
    Reading from a specific meter under a specific contract.
        - client_id tells you who
        - tarif_type and counter_type tell you what kind of service
        - counter_number tells you which device
    So our imputation logic should say:
    “Fill missing meter IDs only when we are confident it’s the same meter under the same conditions.”

 """
data_copy = df.copy()
group_cols = ['client_id', 'tarif_type', 'counter_type']
data_copy['counter_number'] = data_copy.groupby(group_cols)['counter_number'].transform(lambda x: x.ffill().bfill())

data_copy['reading_remarque'] = data_copy['reading_remarque'].fillna('unknown')

data_copy.isnull().sum().sum()


np.int64(0)

In [38]:
data_copy.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667.0,0,203,8.0,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667.0,0,203,6.0,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667.0,0,203,8.0,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667.0,0,207,8.0,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667.0,0,207,9.0,1,572,0,0,0,15066,15638,12,ELEC


In [23]:
#Run a descriptive analysis on numeric features (columns)
print(data_copy[['reading_remarque','invoice_date']].dtypes)

reading_remarque_set = set(data_copy['reading_remarque'].unique())
print(reading_remarque_set)


data_copy[['reading_remarque','consommation_level_1','consommation_level_2','consommation_level_3','consommation_level_4','old_index','new_index']].describe().T

reading_remarque    object
invoice_date        object
dtype: object
{5.0, 6.0, 7.0, 8.0, 9.0, 203.0, 207.0, 'unknown', 413.0}


,count,mean,std,min,25%,50%,75%,max
consommation_level_1,4476749.0,410.979495,757.307998,0.0,79.0,274.0,600.0,999910.0
consommation_level_2,4476749.0,109.322476,1220.123335,0.0,0.0,0.0,0.0,999073.0
consommation_level_3,4476749.0,20.306204,157.423938,0.0,0.0,0.0,0.0,64492.0
consommation_level_4,4476749.0,52.925881,875.472477,0.0,0.0,0.0,0.0,547946.0
old_index,4476749.0,17766.998257,40366.925611,0.0,1791.0,7690.0,21660.0,2800280.0
new_index,4476749.0,18349.704625,40953.206252,0.0,2056.0,8192.0,22343.0,2870972.0


In [27]:
counter_coef_set = set(data_copy['counter_coefficient'].unique())
print(counter_coef_set)

{np.int64(0), np.int64(1), np.int64(33), np.int64(3), np.int64(4), np.int64(2), np.int64(6), np.int64(5), np.int64(40), np.int64(9), np.int64(10), np.int64(11), np.int64(8), np.int64(50), np.int64(20), np.int64(30)}


In [29]:
#Select the bills records for the client with an id ='train_Client_0', using 2 methods.

#Method 1 
client_df = data_copy[data_copy['client_id'] == 'train_Client_0'].copy()

client_df['bill_amount'] = (client_df['new_index'] - client_df['old_index']) * client_df['counter_coefficient'] 
client_df

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,bill_amount
0,train_Client_0,2014-03-24,11,1335667.0,0,203,8.0,1,82,0,0,0,14302,14384,4,ELEC,82
1,train_Client_0,2013-03-29,11,1335667.0,0,203,6.0,1,1200,184,0,0,12294,13678,4,ELEC,1384
2,train_Client_0,2015-03-23,11,1335667.0,0,203,8.0,1,123,0,0,0,14624,14747,4,ELEC,123
3,train_Client_0,2015-07-13,11,1335667.0,0,207,8.0,1,102,0,0,0,14747,14849,4,ELEC,102
4,train_Client_0,2016-11-17,11,1335667.0,0,207,9.0,1,572,0,0,0,15066,15638,12,ELEC,572
5,train_Client_0,2017-07-17,11,1335667.0,0,207,9.0,1,314,0,0,0,15638,15952,8,ELEC,314
6,train_Client_0,2018-12-07,11,1335667.0,0,207,9.0,1,541,0,0,0,15952,16493,12,ELEC,541
7,train_Client_0,2019-03-19,11,1335667.0,0,207,9.0,1,585,0,0,0,16493,17078,8,ELEC,585
8,train_Client_0,2011-07-22,11,1335667.0,0,203,9.0,1,1200,186,0,0,7770,9156,4,ELEC,1386
9,train_Client_0,2011-11-22,11,1335667.0,0,203,6.0,1,1082,0,0,0,9156,10238,4,ELEC,1082


In [31]:
#Method 2 

client_df_query = data_copy.query("client_id == 'train_Client_0'").copy()
client_df_query['bill_amount'] = (client_df_query['new_index'] - client_df_query['old_index']) * client_df_query['counter_coefficient']
client_df_query

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,bill_amount
0,train_Client_0,2014-03-24,11,1335667.0,0,203,8.0,1,82,0,0,0,14302,14384,4,ELEC,82
1,train_Client_0,2013-03-29,11,1335667.0,0,203,6.0,1,1200,184,0,0,12294,13678,4,ELEC,1384
2,train_Client_0,2015-03-23,11,1335667.0,0,203,8.0,1,123,0,0,0,14624,14747,4,ELEC,123
3,train_Client_0,2015-07-13,11,1335667.0,0,207,8.0,1,102,0,0,0,14747,14849,4,ELEC,102
4,train_Client_0,2016-11-17,11,1335667.0,0,207,9.0,1,572,0,0,0,15066,15638,12,ELEC,572
5,train_Client_0,2017-07-17,11,1335667.0,0,207,9.0,1,314,0,0,0,15638,15952,8,ELEC,314
6,train_Client_0,2018-12-07,11,1335667.0,0,207,9.0,1,541,0,0,0,15952,16493,12,ELEC,541
7,train_Client_0,2019-03-19,11,1335667.0,0,207,9.0,1,585,0,0,0,16493,17078,8,ELEC,585
8,train_Client_0,2011-07-22,11,1335667.0,0,203,9.0,1,1200,186,0,0,7770,9156,4,ELEC,1386
9,train_Client_0,2011-11-22,11,1335667.0,0,203,6.0,1,1082,0,0,0,9156,10238,4,ELEC,1082


In [35]:
print(set(data_copy['counter_type'].unique()))

{'GAZ', 'ELEC'}


In [39]:
#Transform the 'counter_type' feature to a numeric variable using the encoder of your choice



encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(data_copy[['counter_type']])

one_hot_data_copy = pd.DataFrame(one_hot_encoded, 
                          columns=encoder.get_feature_names_out(['counter_type']))

df_sklearn_encoded = pd.concat([df.drop(['counter_type'], axis=1), one_hot_data_copy], axis=1)

df_sklearn_encoded

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type_ELEC,counter_type_GAZ
0,train_Client_0,2014-03-24,11,1335667.0,0,203,8.0,1,82,0,0,0,14302,14384,4,1.0,0.0
1,train_Client_0,2013-03-29,11,1335667.0,0,203,6.0,1,1200,184,0,0,12294,13678,4,1.0,0.0
2,train_Client_0,2015-03-23,11,1335667.0,0,203,8.0,1,123,0,0,0,14624,14747,4,1.0,0.0
3,train_Client_0,2015-07-13,11,1335667.0,0,207,8.0,1,102,0,0,0,14747,14849,4,1.0,0.0
4,train_Client_0,2016-11-17,11,1335667.0,0,207,9.0,1,572,0,0,0,15066,15638,12,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4476744,train_Client_99998,2005-08-19,10,1253571.0,0,202,9.0,1,400,135,0,0,3197,3732,8,1.0,0.0
4476745,train_Client_99998,2005-12-19,10,1253571.0,0,202,6.0,1,200,6,0,0,3732,3938,4,1.0,0.0
4476746,train_Client_99999,1996-09-25,11,560948.0,0,203,6.0,1,259,0,0,0,13884,14143,4,1.0,0.0
4476747,train_Client_99999,1996-05-28,11,560948.0,0,203,6.0,1,603,0,0,0,13281,13884,4,1.0,0.0


In [40]:
#Delete
data_copy.drop('counter_statue', axis=1, inplace=True)
data_copy

,client_id,invoice_date,tarif_type,counter_number,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667.0,203,8.0,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667.0,203,6.0,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667.0,203,8.0,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667.0,207,8.0,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667.0,207,9.0,1,572,0,0,0,15066,15638,12,ELEC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4476744,train_Client_99998,2005-08-19,10,1253571.0,202,9.0,1,400,135,0,0,3197,3732,8,ELEC
4476745,train_Client_99998,2005-12-19,10,1253571.0,202,6.0,1,200,6,0,0,3732,3938,4,ELEC
4476746,train_Client_99999,1996-09-25,11,560948.0,203,6.0,1,259,0,0,0,13884,14143,4,ELEC
4476747,train_Client_99999,1996-05-28,11,560948.0,203,6.0,1,603,0,0,0,13281,13884,4,ELEC
